In [20]:
!pip install transformers -q
!pip install accelerate>=0.26.0 -q


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np

class Agent:
    def __init__(self, name, role, model_name, device):
        self.name = name
        self.role = role  # Specialization
        self.model_name = model_name
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        ).to(device)
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if device == 'cuda' else -1,
            max_length=2048,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0  # Initial weight

    def generate_response(self, messages):
        # Combine messages into a single prompt
        prompt = ""
        for message in messages:
            if message['role'] == 'system':
                prompt += f"System: {message['content']}\n"
            elif message['role'] == 'user':
                prompt += f"User: {message['content']}\n"
            elif message['role'] == 'assistant':
                prompt += f"Assistant: {message['content']}\n"
        prompt += "Assistant:"

        # Generate response
        response = self.pipeline(prompt, max_length=2048, num_return_sequences=1)[0]['generated_text']
        # Extract the assistant's response
        response = response[len(prompt):].strip().split('User:')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)  # Prevent negative weights

def construct_message(agents, idx, question, round_num, final=False, self_reflection=False):
    agent = agents[idx]
    messages = []
    # System prompt with agent's role
    system_prompt = f"You are an AI assistant specialized in {agent.role}. Assist with the following task."
    messages.append({'role': 'system', 'content': system_prompt})

    # Self-reflection
    if self_reflection and len(agent.history) > 0:
        last_response = agent.history[-1]['response']
        reflection_prompt = (
            f"Review your previous response: '{last_response}'. "
            f"Is there any improvement or correction you would like to make?"
        )
        messages.append({'role': 'user', 'content': reflection_prompt})

    # Include other agents' weighted responses
    if round_num > 0 and not self_reflection:
        summary = "Here are the key points from other agents:\n"
        total_score = sum(agent.score for agent in agents if agent != agents[idx])
        for other_agent in agents:
            if other_agent != agents[idx]:
                weight = other_agent.score / total_score if total_score > 0 else 0
                weighted_response = (
                    f"Agent {other_agent.name} (Weight: {weight:.2f}): {other_agent.history[-1]['response']}\n"
                )
                summary += weighted_response
        messages.append({'role': 'user', 'content': summary})

    # Main prompt
    if not final and not self_reflection:
        prompt = (
            f"Provide a step-by-step reasoning and answer to the question: {question}"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    messages.append({'role': 'user', 'content': prompt})

    return messages

def run_debate(agents, question, confidence_threshold=0.8):
    consensus_reached = False
    round_num = 0
    max_rounds = 5
    while not consensus_reached and round_num < max_rounds:
        print(f"\n=== Round {round_num + 1} ===")
        # Agents generate responses
        for idx, agent in enumerate(agents):
            messages = construct_message(agents, idx, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'messages': messages, 'response': response})
            print(f"\nAgent {agent.name} ({agent.role}) response:\n{response}\n")

        # Agents perform self-reflection
        for idx, agent in enumerate(agents):
            messages = construct_message(agents, idx, question, round_num, self_reflection=True)
            reflection_response = agent.generate_response(messages)
            agent.history[-1]['response'] = reflection_response  # Update with reflection
            print(f"Agent {agent.name} after self-reflection:\n{reflection_response}\n")

        # Evaluate consensus
        consensus_score = calculate_consensus([agent.history[-1]['response'] for agent in agents])
        print(f"Consensus score: {consensus_score:.2f}")
        if consensus_score >= confidence_threshold:
            consensus_reached = True
            print("Consensus reached among agents.")
        else:
            round_num += 1

    # Agents provide final answers
    print("\n=== Final Answers ===")
    for idx, agent in enumerate(agents):
        messages = construct_message(agents, idx, question, round_num, final=True)
        final_response = agent.generate_response(messages)
        agent.history.append({'messages': messages, 'response': final_response})
        print(f"\nAgent {agent.name} ({agent.role}) final response:\n{final_response}\n")

def calculate_consensus(responses):
    # Simple consensus calculation based on response similarity
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score



In [11]:
from huggingface_hub import login
import os

# Set your Hugging Face API token
# Replace 'your_token_here' with your actual token
os.environ["HUGGINGFACE_TOKEN"] = "your_token_here"

# Login
login()

In [37]:
!pip install 'accelerate>=0.26.0'


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [38]:
# Device configuration: use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define models for each agent based on their specialization
agents = [
    # Agent(
    #     name='Agent1',
    #     role='mathematical reasoning',
    #     model_name='meta-llama/Llama-3.2-1b',
    #     device=device
    # ),
    Agent(
        name='Agent2',
        role='logical consistency',
        model_name='Qwen/Qwen2.5-7B-Instruct',
        device=device
    ),
    Agent(
        name='Agent3',
        role='context integration',
        model_name='deepseek-ai/DeepSeek-V2.5',
        device=device
    ),
]

question = "What is the sum of all even numbers between 1 and 100?"

run_debate(agents, question)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`